In [2]:
import sys
from collections import deque
import numpy as np
import theano
import theano.tensor as T
import lasagne
sys.path.append("..")
from qtable import QAgent, SimpleMaze
from qnn import QAgentNN, wrap_as_tensor3

In [3]:
maze = SimpleMaze()
agent = QAgentNN(dim_state=(1, 1, 2), range_state=((((0, 3),(0, 4)),),),actions=maze.actions,
                 learning_rate=0.01, reward_scaling=100,
                 freeze_period=100,
                 alpha=0.5, gamma=0.5, explore_strategy='fixed_epsilon', epsilon=0.2)

In [4]:
maze = SimpleMaze()
agent = QAgentNN(dim_state=(1, 1, 2), range_state=((((0, 3),(0, 4)),),),actions=maze.actions,
                 learning_rate=0.01, reward_scaling=100, batch_size=100,
                 freeze_period=100, memory_size=1000,
                 alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.02)
print "Maze and agent initialized!"

# logging
path = deque()  # path in this episode
episode_reward_rates = []
num_episodes = 0
cum_reward = 0
cum_steps = 0

# repeatedly run episodes
while True:
    maze.reset()
    new_observation = maze.observe()
    agent.reset()

    path.clear()
    path.append(new_observation)
    episode_reward = 0
    episode_steps = 0
    episode_loss = 0

    # interact and reinforce repeatedly
    while not maze.isfinished():
        action = agent.act(wrap_as_tensor3(new_observation))
        new_observation, reward = maze.interact(action)
        loss = agent.reinforce(current_observation=wrap_as_tensor3(new_observation), reward=reward)
        # print action,
        # print new_observation,
        path.append(new_observation)
        episode_reward += reward
        episode_steps += 1
        episode_loss += loss if loss else 0
    print len(path),
    # print "{:.3f}".format(episode_loss),
    # print ""
    cum_steps += episode_steps
    cum_reward += episode_reward
    num_episodes += 1
    episode_reward_rates.append(episode_reward / episode_steps)
    if num_episodes % 100 == 0:
        print ""
        print num_episodes, cum_reward, cum_steps, 1.0 * cum_reward / cum_steps #, path
        cum_reward = 0
        cum_steps = 0
win = 50
s = pd.rolling_mean(pd.Series([0]*win+episode_reward_rates), window=win, min_periods=1)
s.plot()
plt.show()

Maze and agent initialized!
1483 1114 4 7 645 34 2 3 5 2 2 3 291 303 3 99 92 2 3 89 3 3 4 34 3 4 46 3 3 6 4 5 4 41 3 3 4 4 5 4 3 6 3 3 4 4 3 3 4 3 3 4 5 3 3 4 2 5 5 5 5 5 4 3 4 4 3 5 3 4 3 2 2 2 2 2 5 3 3 2 4 4 4 4 3 3 5 4 3 4 4 5 4 2 4 4 4 6 5 5 
100 8386 4494 1.86604361371
3 4 5 5 4 3 5 3 4 3 4 4 3 2 3 4 3 4 3 4 5 4 3 3 2 5 3 4 3 4 3 3 4 4 3 4 4 3 3 2 3 4 4 3 4 2 3 5 5 4 4 4 2 3 2 4 3 2 2 4 3 3 5 3 3 5 5 4 2 3 2 4 2 3 3 4 3 2 4 4 2 4 4 4 4 3 4 3 4 4 5 3 3 4 3 3 4 2 3 3 
200 10000 245 40.8163265306
5 4 3 6 3 2 4 2 2 5 3 4 3 3 3 2 2 5 5 4 5 2 2 2 4 5 3 3 2 3 2 5 2 4 3 5 2 2 3 4 3 5 3 4 3 2 4 4 2 4 2 3 2 2 2 3 2 4 4 3 3 4 5 2 3 4 3 5 6 4 3 3 2 3 5 5 3 3 3 4 5 4 3 2 4 3 2 4 4 5 3 3 2 4 5 3 5 4 4 3 
300 9999 239 41.8368200837
4 4 3 2 2 3 3 5 3 3 3 4 2 3 5 4 2 5 3 5 3 3 2 2 3 2 2 6 3 4 6 3 4 3 5 4 3 3 4 4 3 4 4 2 4 4 2 3 3 3 4 2 2 3 3 3 2 4 3 3 3 4 2 4 3 3 3 2 3 3 2 4 3 4 5 4 3 4 5 3 2 3 4 4 5 3 4 4 3 3 2 3 2 3 3 3 5 3 2 3 
400 9999 229 43.6637554585
4 5 4 2 4 3 4 3 5 4 4 2 4 4 3 4 3 4 4 3

KeyboardInterrupt: 